## <p style="background-color:#757D70;color:#D5CABD;font-size:110%;text-align:center;border-radius:20px 60px;  border: 10px solid #cfcfcf;  padding-top: 10px;  padding-left: 10px;  padding-right: 10px; padding-bottom: 10px;  border-radius: 80px 20px;line-height: 1.2em; margin-bottom: 10px;"> **Partie** :Règles d'association APRIORI   </p>

---
- Groupe 2
- Professeur : [**Sayoba GANSANE**]()
- Ecole : [**Université Alioune Diop de Bambey (UADB)**]()
- Projet: [**Data Mining Approche Informatique**]()
- Niveau :  [**Master 2 : Statistique et Informatique Decisionnel (SID)**]()



---

### Règles d’association avec l’algorithme APRIORI

**Objectif :**  
Mettre en évidence les associations fréquentes entre les produits achetés par les clients
afin d’identifier des opportunités de ventes croisées (cross-selling).

L’algorithme APRIORI permet d’extraire des règles d’association basées sur :
- le support,
- la confiance,
- le lift.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimpy

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder


### Chargement des données

Les données utilisées ont été préalablement nettoyées lors de l’analyse descriptive.
Elles contiennent uniquement des transactions valides.


In [15]:
data = pd.read_csv("donnees_ecommerce_netoyer.csv", encoding="latin-1")

In [3]:
df= data.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Prix,Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12


In [4]:
skimpy.skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 392692 │ │ string      │ 5     │                                                          │
│ │ Number of columns │ 10     │ │ float64     │ 3     │                                                          │
│ └───────────────────┴────────┘ │ int64       │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column       ┃ NA  ┃ NA %  ┃ mean     ┃ sd     ┃ p0      ┃ p25     ┃ p50     ┃ p75     ┃ p100    ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩  │
│ │ InvoiceNo    │   0 │     0 │   560600 │  13090 │  536400 │  549200 │  561900 │  572100 │  581600 │ ▆▆▆▇██  │  │
│ │ Quantity     │   0 │     0 │    13.12 │  180.5 │       1 │       2 │       6 │      12 │   81000 │    █    │  │
│ │ UnitPrice    │   0 │     0 │    3.126 │  22.24 │   0.001 │    1.25 │    1.95 │    3.75 │    8143 │    █    │  │
│ │ CustomerID   │   0 │     0 │    15290 │   1714 │   12350 │   13960 │   15150 │   16790 │   18290 │ ▇▆█▆▆▇  │  │
│ │ Total_Prix   │   0 │     0 │    22.63 │  311.1 │   0.001 │    4.95 │   12.45 │    19.8 │  168500 │    █    │  │
│ └──────────────┴─────┴───────┴──────────┴────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓  │
│ ┃           ┃    ┃      ┃           ┃           ┃           ┃           ┃ chars per ┃ words per ┃ total      ┃  │
│ ┃ column    ┃ NA ┃ NA % ┃ shortest  ┃ longest   ┃ min       ┃ max       ┃ row       ┃ row       ┃ words      ┃  │
│ ┡━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩  │
│ │ StockCode │  0 │    0 │ M         │ BANK      │ 10002     │ POST      │      5.08 │         1 │     392704 │  │
│ │           │    │      │           │ CHARGES   │           │           │           │           │            │  │
│ │ Descripti │  0 │    0 │ Manual    │ KNITTED   │  4 PURPLE │ ZINC WIRE │      26.7 │       4.7 │    1826450 │  │
│ │ on        │    │      │           │ UNION     │ FLOCK     │ SWEETHEAR │           │           │            │  │
│ │           │    │      │           │ FLAG HOT  │ DINNER    │ T LETTER  │           │           │            │  │
│ │           │    │      │           │ WATER     │ CANDLES   │ TRAY      │           │           │            │  │
│ │           │    │      │           │ BOTTLE    │           │           │           │           │            │  │
│ │ InvoiceDa │  0 │    0 │ 2010-12-0 │ 2010-12-0 │ 2010-12-0 │ 2011-12-0 │        19 │         2 │     785384 │  │
│ │ te        │    │      │ 1         │ 1         │ 1         │ 9         │           │           │            │  │
│ │           │    │      │ 08:26:00  │ 08:26:00  │ 08:26:00  │ 12:50:00  │           │           │            │  │
│ │ Country   │  0 │    0 │ USA       │ United    │ Aust

### Sélection des variables

Pour l’analyse des règles d’association, seules les informations suivantes sont nécessaires :
- identifiant de la transaction,
- description du produit.


In [5]:
df_apriori = df[["InvoiceNo", "Description"]]
df_apriori.head()


,InvoiceNo,Description
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.


### Construction des paniers d’achat

Chaque transaction est transformée en une liste de produits achetés.
Cette structure est indispensable pour appliquer l’algorithme APRIORI.


In [6]:
transactions = (
    df_apriori.groupby("InvoiceNo")["Description"]
    .apply(list)
    .tolist()
)

transactions[:5]


[['WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN',
  'CREAM CUPID HEARTS COAT HANGER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'GLASS STAR FROSTED T-LIGHT HOLDER'],
 ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'],
 ['ASSORTED COLOUR BIRD ORNAMENT',
  "POPPY'S PLAYHOUSE BEDROOM ",
  "POPPY'S PLAYHOUSE KITCHEN",
  'FELTCRAFT PRINCESS CHARLOTTE DOLL',
  'IVORY KNITTED MUG COSY ',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BOX OF VINTAGE JIGSAW BLOCKS ',
  'BOX OF VINTAGE ALPHABET BLOCKS',
  'HOME BUILDING BLOCK WORD',
  'LOVE BUILDING BLOCK WORD',
  'RECIPE BOX WITH METAL HEART',
  'DOORMAT NEW ENGLAND'],
 ['JAM MAKING SET WITH JARS',
  'RED COAT RACK PARIS FASHION',
  'YELLOW COAT RACK PARIS FASHION',
  'BLUE COAT RACK PARIS FASHION'],
 ['BATH BUILDING BLOCK WORD']]

### Encodage des transactions

Les paniers d’achat sont transformés en une matrice binaire (one-hot encoding),
où chaque colonne représente un produit et chaque ligne une transaction.


In [7]:
from mlxtend.preprocessing import TransactionEncoder

# On part de ton DataFrame 'data'
transactions = data.values.tolist()


In [8]:
# Nettoyer les transactions : supprimer les NaN et convertir en str
transactions = [
    [str(item) for item in transaction if pd.notna(item)]
    for transaction in transactions
]

In [9]:
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Total_Prix', 'Month'],
      dtype='object')


In [10]:
item_freq = df['Description'].value_counts(normalize=True)

items_frequents = item_freq[item_freq >= 0.01].index

df_filtre = df[df['Description'].isin(items_frequents)]

transactions = (
    df_filtre
    .groupby('InvoiceNo')['Description']
    .apply(list)
)


In [11]:
transactions.head()


Series([], Name: Description, dtype: object)

In [12]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Total_Prix', 'Month'],
      dtype='object')